In [1]:
import torch
import pandas as pd
import numpy as np
import torchio as tio
from torch.utils.data import DataLoader
import os
import SimpleITK as sitk
from zipfile import ZipFile
from zipfile import BadZipFile
import dask.dataframe as dd
import os
import multiprocessing as mp
import functools
from functools import partial
import Standardize


## managment of files 
managment of files is done via managePicaiFiles.sh - create directories download and unpack files saves basic metadata and do simple metadata preprocessing
sh managePicaiFiles.sh

## Standarization
primary preprocessing - removing ouliers, put values between 0 and 255 bias field correction Nyul standarization binarizing labels

In [2]:
df = pd.read_csv('/home/sliceruser/data/metadata/processedMetaData.csv')
df['sag']

0       /home/sliceruser/data/orig/10000/10000_1000000...
1       /home/sliceruser/data/orig/10001/10001_1000001...
2       /home/sliceruser/data/orig/10002/10002_1000002...
3       /home/sliceruser/data/orig/10003/10003_1000003...
4       /home/sliceruser/data/orig/10004/10004_1000004...
                              ...                        
1495    /home/sliceruser/data/orig/11471/11471_1001495...
1496    /home/sliceruser/data/orig/11472/11472_1001496...
1497    /home/sliceruser/data/orig/11473/11473_1001497...
1498    /home/sliceruser/data/orig/11474/11474_1001498...
1499    /home/sliceruser/data/orig/11475/11475_1001499...
Name: sag, Length: 1500, dtype: object

In [3]:
import Standardize
df = pd.read_csv('/home/sliceruser/data/metadata/processedMetaData.csv')
numRows=10
Standardize.iterateAndchangeLabelToOnes(numRows,df)
for keyWord in ['t2w','adc', 'cor','hbv','sag'  ]:
    Standardize.iterateAndStandardize(keyWord,numRows,df)

df.to_csv('/home/sliceruser/data/metadata/processedMetaData.csv') 


## Getting all of the spacings 

In [4]:

import matplotlib.pyplot as plt
import numpy as np
# if using a Jupyter notebook, includue:
%matplotlib inline


In [5]:
spacingDict={}
for keyWord in ['t2w','adc', 'cor','hbv','sag'  ]: 
    for addedKey in ['_spac_x','_spac_y','_spac_z']:   

        colName = keyWord+addedKey
        minn=np.min(df[colName].to_numpy())                
        maxx=np.max(df[colName].to_numpy())
        spacingDict[colName]=(minn,maxx)

spacingDict



{'t2w_spac_x': (0.234375, 0.78125),
 't2w_spac_y': (0.234375, 0.78125),
 't2w_spac_z': (2.200000060372773, 5.000000066297424),
 'adc_spac_x': (0.234375, 0.78125),
 'adc_spac_y': (0.234375, 0.78125),
 'adc_spac_z': (2.200000060372773, 5.000000066297424),
 'cor_spac_x': (0.234375, 0.78125),
 'cor_spac_y': (0.234375, 0.78125),
 'cor_spac_z': (2.200000060372773, 5.000000066297424),
 'hbv_spac_x': (0.234375, 0.78125),
 'hbv_spac_y': (0.234375, 0.78125),
 'hbv_spac_z': (2.200000060372773, 5.000000066297424),
 'sag_spac_x': (0.234375, 0.78125),
 'sag_spac_y': (0.234375, 0.78125),
 'sag_spac_z': (2.200000060372773, 5.000000066297424)}

In [6]:
minXSpac = spacingDict['t2w_spac_x'][0]
minYSpac = spacingDict['t2w_spac_y'][0]
minZSpac = spacingDict['t2w_spac_z'][0]
spacingDict['t2w_spac_z']

(2.200000060372773, 5.000000066297424)

In [7]:
# from KevinSR import mask_interpolation, SOUP_GAN
# import os
# import numpy as np
# from scipy import ndimage, interpolate
# from scipy.ndimage import zoom
# from KevinSR import SOUP_GAN
# import monai

# import SimpleITK as sitk

# path = df['t2w'].to_numpy()[0]



# spacingX_a= 0.234375
# spacingX_b= 0.78125

# spacingY_a= 0.234375
# spacingY_b= 0.78125

# spacingZ_a= 2.200000060372773
# spacingZ_b= 5.000000066297424

# img = sitk.ReadImage(path)
# image1 = sitk.ReadImage(path)

# thicks_ori = sitk.GetArrayFromImage(image1)
# Z_FAC = spacingZ_b/spacingZ_a

# # Call the SR interpolation tool from KevinSR
# thins_gen = SOUP_GAN(thicks_ori, Z_FAC, 0)




# #recreating image keeping relevant metadata
# image = sitk.GetImageFromArray(data)
# image.SetSpacing(image1.GetSpacing())
# image.SetOrigin(image1.GetOrigin())
# image.SetDirection(image1.GetDirection())



# #Example of thick-to-thin type of preprocessing
# pre_slices = 27
# post_slices = 135


# # thicks_ori = load_data(DIR_dicom)
# # Z_FAC = post_slices/pre_slices # Sampling factor in Z direction

# # thicks_ori = rescale_img(thicks_ori, max_val= 10000)

# # thins = zoom(thicks_ori, (1,1,Z_FAC))
# # thins_raw = zoom(thicks_ori, (1,1,Z_FAC),order=0)

# # # Call the SR interpolation tool from KevinSR
# # thins_gen = SOUP_GAN(thicks_ori, Z_FAC, 0)

# # # Plot the original thick slices, thin slices by TCI and SR generated slices. 
# # plot_scans(thins_raw, thins, thins_gen, 0)


In [8]:
# for keyWord in ['t2w','adc', 'cor','hbv','sag'  ]: 
#     for addedKey in ['_sizz_x','_sizz_y','_sizz_z','_spac_x'
#                         ,'_spac_y','_spac_z'
#                         ,'_orig_x','_orig_x','_orig_x']:  